<a href="https://colab.research.google.com/github/bhargav23/AI/blob/master/UNIT6/Content_based_Filtering_Recommender_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender Systems : Content Based systems

# Know Your Data

In [1]:
# Data for content based recommendations
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/bhargav23/Dataset/master/Content_file_IMDB.csv')
df.head(2) # display top two rows of the data

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True


In [2]:
df.columns

Index(['Unnamed: 0', 'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre',
       'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards',
       'Poster', 'Ratings.Source', 'Ratings.Value', 'Metascore', 'imdbRating',
       'imdbVotes', 'imdbID', 'Type', 'tomatoMeter', 'tomatoImage',
       'tomatoRating', 'tomatoReviews', 'tomatoFresh', 'tomatoRotten',
       'tomatoConsensus', 'tomatoUserMeter', 'tomatoUserRating',
       'tomatoUserReviews', 'tomatoURL', 'DVD', 'BoxOffice', 'Production',
       'Website', 'Response'],
      dtype='object')

In [0]:
df = df[['Title','Genre','Director','Actors','Plot']]

In [4]:
df

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...
...,...,...,...,...,...
245,The Lost Weekend,"Drama, Film-Noir",Billy Wilder,"Ray Milland, Jane Wyman, Phillip Terry, Howard...",The desperate life of a chronic alcoholic is f...
246,Short Term 12,Drama,Destin Daniel Cretton,"Brie Larson, John Gallagher Jr., Stephanie Bea...",A 20-something supervising staff member of a r...
247,His Girl Friday,"Comedy, Drama, Romance",Howard Hawks,"Cary Grant, Rosalind Russell, Ralph Bellamy, G...",A newspaper editor uses every trick in the boo...
248,The Straight Story,"Biography, Drama",David Lynch,"Sissy Spacek, Jane Galloway Heitz, Joseph A. C...",An old man makes a long journey by lawn-mover ...


In [5]:
!pip install rake_nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=6a9e2efc913188af0703d6a11e0e3eadfeebc70a75cc91d1daa201188d4c6ddc
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import rake_nltk
from rake_nltk import Rake

In [7]:
df['Actors'][0]

'Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler'

In [0]:
# discarding the commas between the actors' full names and getting only the first three names
df['Actors'] = df['Actors'].map(lambda x: x.split(',')[:3])

In [9]:
df['Actors'][0]

['Tim Robbins', ' Morgan Freeman', ' Bob Gunton']

In [10]:
df['Genre'][0]

'Crime, Drama'

In [0]:
# putting the genres in a list of words
df['Genre'] = df['Genre'].map(lambda x: x.lower().split(','))

In [12]:
df['Genre'][0]

['crime', ' drama']

In [13]:
df['Director'][0]

'Frank Darabont'

In [0]:
df['Director'] = df['Director'].map(lambda x: x.split(' '))

In [15]:
df['Director'][0]

['Frank', 'Darabont']

In [0]:
# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in df.iterrows():
    row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    row['Director'] = ''.join(row['Director']).lower()

In [17]:
df['Plot'][0]

'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'

In [18]:
df['Plot'][1]

'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.'

In [0]:
r = Rake()

In [0]:
r.extract_keywords_from_text(df['Plot'][0])

In [0]:
d = r.get_word_degrees()

In [22]:
d

defaultdict(<function rake_nltk.rake.Rake._build_word_co_occurance_graph.<locals>.<lambda>>,
            {'acts': 1,
             'bond': 4,
             'common': 2,
             'decency': 2,
             'eventual': 2,
             'finding': 2,
             'imprisoned': 4,
             'men': 4,
             'number': 1,
             'redemption': 2,
             'solace': 2,
             'two': 4,
             'years': 1})

In [23]:
list(d.keys())

['common',
 'decency',
 'finding',
 'solace',
 'acts',
 'two',
 'imprisoned',
 'men',
 'bond',
 'eventual',
 'redemption',
 'years',
 'number']

Extracting the key words from the plot description to make movie profiles. Text preprocessing.

In [0]:
# New column to store the keywords of the plot
df['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['Plot']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
# df.drop(columns = ['Plot'], inplace = True)
df.drop('Plot',axis = 1, inplace = True)
df.set_index('Title', inplace = True)

In [0]:
columns = df.columns

In [26]:
columns

Index(['Genre', 'Director', 'Actors', 'Key_words'], dtype='object')

In [0]:
# constructing bag of words vectors for movies
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Director': # only one word in the column
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    


In [28]:
df['bag_of_words'][0]

'crime  drama frankdarabont timrobbins morganfreeman bobgunton common decency finding solace acts two imprisoned men bond eventual redemption years number  '

In [29]:
df

,Genre,Director,Actors,Key_words,bag_of_words
Title,,,,,
The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton]","[common, decency, finding, solace, acts, two, ...",crime drama frankdarabont timrobbins morganfr...
The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan]","[reluctant, son, clandestine, empire, organize...",crime drama francisfordcoppola marlonbrando a...
The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton]","[son, 1920s, new, york, grip, vito, corleone, ...",crime drama francisfordcoppola alpacino rober...
The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart]","[joker, emerges, mysterious, past, physical, t...",action crime drama christophernolan christia...
12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]","[miscarriage, justice, evidence, jury, holdout...",crime drama sidneylumet martinbalsam johnfied...
...,...,...,...,...,...
The Lost Weekend,"[drama, film-noir]",billywilder,"[raymilland, janewyman, phillipterry]","[day, drinking, bout, four, chronic, alcoholic...",drama film-noir billywilder raymilland janewy...
Short Term 12,[drama],destindanielcretton,"[brielarson, johngallagherjr., stephaniebeatriz]","[world, alongside, 20, worker, something, supe...",drama destindanielcretton brielarson johngalla...
His Girl Friday,"[comedy, drama, romance]",howardhawks,"[carygrant, rosalindrussell, ralphbellamy]","[newspaper, editor, uses, every, trick, wife, ...",comedy drama romance howardhawks carygrant r...


In [30]:
df.drop(columns=['Genre','Director','Actors','Key_words'])

,bag_of_words
Title,
The Shawshank Redemption,crime drama frankdarabont timrobbins morganfr...
The Godfather,crime drama francisfordcoppola marlonbrando a...
The Godfather: Part II,crime drama francisfordcoppola alpacino rober...
The Dark Knight,action crime drama christophernolan christia...
12 Angry Men,crime drama sidneylumet martinbalsam johnfied...
...,...
The Lost Weekend,drama film-noir billywilder raymilland janewy...
Short Term 12,drama destindanielcretton brielarson johngalla...
His Girl Friday,comedy drama romance howardhawks carygrant r...


In [31]:
df['bag_of_words'][0]

'crime  drama frankdarabont timrobbins morganfreeman bobgunton common decency finding solace acts two imprisoned men bond eventual redemption years number  '

Obtaining the representations for movies

In [0]:
#Construct content based feature vectors for items(movies)
# Solution


from sklearn.feature_extraction.text import TfidfVectorizer
# instantiating and generating the count matrix
count = TfidfVectorizer()

count_matrix = count.fit_transform(df['bag_of_words'])



In [33]:
count_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
df.index

Index(['The Shawshank Redemption', 'The Godfather', 'The Godfather: Part II',
       'The Dark Knight', '12 Angry Men', 'Schindler's List',
       'The Lord of the Rings: The Return of the King', 'Pulp Fiction',
       'Fight Club', 'The Lord of the Rings: The Fellowship of the Ring',
       ...
       'Deadpool', 'The Wild Bunch', 'Aladdin', 'Big Fish', 'Patton',
       'The Lost Weekend', 'Short Term 12', 'His Girl Friday',
       'The Straight Story', 'Slumdog Millionaire'],
      dtype='object', name='Title', length=250)

In [35]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df.index)
indices

0      The Shawshank Redemption
1                 The Godfather
2        The Godfather: Part II
3               The Dark Knight
4                  12 Angry Men
                 ...           
245            The Lost Weekend
246               Short Term 12
247             His Girl Friday
248          The Straight Story
249         Slumdog Millionaire
Name: Title, Length: 250, dtype: object

In [36]:
# generating the cosine similarity matrix
# Solution  X = count_matrix
cosine_sim = cosine_similarity(count_matrix)
cosine_sim

array([[1.        , 0.02996184, 0.02611648, ..., 0.00386164, 0.00383188,
        0.00372325],
       [0.02996184, 1.        , 0.1914016 , ..., 0.00420721, 0.00417478,
        0.00405643],
       [0.02611648, 0.1914016 , 1.        , ..., 0.00366725, 0.00363898,
        0.00353582],
       ...,
       [0.00386164, 0.00420721, 0.00366725, ..., 1.        , 0.00390566,
        0.00379494],
       [0.00383188, 0.00417478, 0.00363898, ..., 0.00390566, 1.        ,
        0.00376569],
       [0.00372325, 0.00405643, 0.00353582, ..., 0.00379494, 0.00376569,
        1.        ]])

In [0]:
idx = indices[indices == "The Lost Weekend"].index[0]

In [38]:
idx

245

In [39]:
cosine_sim[245]

array([0.0042977 , 0.00468229, 0.02857501, 0.00351536, 0.00442689,
       0.00400306, 0.00398325, 0.04282835, 0.02456941, 0.00399694,
       0.0041732 , 0.        , 0.        , 0.00372963, 0.0040339 ,
       0.02615978, 0.        , 0.        , 0.00448318, 0.03024013,
       0.00403042, 0.00368594, 0.00380775, 0.00420721, 0.00378762,
       0.00486285, 0.00448111, 0.00474581, 0.00452007, 0.00417464,
       0.        , 0.        , 0.00455449, 0.        , 0.00468577,
       0.00362755, 0.        , 0.        , 0.00443041, 0.00368037,
       0.00455799, 0.003973  , 0.        , 0.00419938, 0.14453036,
       0.0199337 , 0.        , 0.00444679, 0.00446293, 0.00385224,
       0.05696648, 0.        , 0.        , 0.03171629, 0.02677893,
       0.        , 0.00444393, 0.        , 0.        , 0.        ,
       0.00388271, 0.00437477, 0.11211661, 0.0474752 , 0.00385261,
       0.00508137, 0.00306857, 0.0039959 , 0.00413865, 0.00425041,
       0.02872663, 0.00342491, 0.00397334, 0.04796303, 0.     

In [0]:
ss = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

In [41]:
ss

245    1.000000
44     0.144530
116    0.132225
62     0.112117
180    0.084069
         ...   
186    0.000000
185    0.000000
33     0.000000
223    0.000000
133    0.000000
Length: 250, dtype: float64

In [42]:
ss.iloc[1:11].index

Int64Index([44, 116, 62, 180, 218, 220, 164, 170, 191, 150], dtype='int64')

In [0]:
# Compute similarity between the items to recommend similar items.

# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [44]:
recommendations('12 Angry Men') 

['On the Waterfront',
 'The Exorcist',
 'American History X',
 'Dog Day Afternoon',
 'L.A. Confidential',
 'Network',
 'The Nightmare Before Christmas',
 'A Christmas Story',
 'Logan',
 'The Godfather']

In [45]:
# print(indices)
# movieid 16 is the movie The Matrix
recommendations('The Matrix') 

['2001: A Space Odyssey',
 'Memento',
 'Guardians of the Galaxy Vol. 2',
 'Mad Max: Fury Road',
 'The Terminator',
 'Alien',
 'Guardians of the Galaxy',
 'How to Train Your Dragon',
 'Aliens',
 'A Streetcar Named Desire']

In [46]:
recommendations('The Godfather') 

['The Godfather: Part II',
 'Apocalypse Now',
 'On the Waterfront',
 'Scarface',
 'The Night of the Hunter',
 'Casino',
 'Heat',
 'Guardians of the Galaxy',
 'All About Eve',
 "Who's Afraid of Virginia Woolf?"]